# scRNAseq comprehensive analysis – 1.Scanpy QC

Import and define functions

In [ ]:
### warning
import warnings
warnings.filterwarnings('ignore')

### Scientific/plotting stack
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
import colorcet as cc
import itertools
from pathlib import Path
from scipy.stats import median_abs_deviation
from scipy.sparse import csr_matrix, issparse
### harmony
import harmonypy as hm
### Scanpy e AnnData
import scanpy as sc
import anndata2ri
import anndata
### decoupler
import decoupler as dc

### celltypist
import celltypist
from celltypist import models

### Graphics
from adjustText import adjust_text
import matplotlib.patheffects as pe

### Logging
import logging

### RPy2 
import rpy2.robjects as ro
import rpy2.rinterface_lib.callbacks as rcb
from rpy2.robjects import r
from rpy2.robjects.packages import importr

import hdf5plugin
anndata2ri.activate()


%load_ext rpy2.ipython
# Remember to insert in <username> your pc username 
os.makedirs("Users/<username>/results/info_Plots/Harmony", exist_ok=True)
os.makedirs("Users/<username>/results/info_Plots/Umap", exist_ok=True)
os.makedirs("Users/<username>/results/Markers_Plots/Harmony", exist_ok=True)
os.makedirs("Users/<username>/results/Markers_Plots/Umap", exist_ok=True)
os.makedirs("Users/<username>/results/Markers_Plots/PreFiltering", exist_ok=True)
os.makedirs("Users/<username>/results/Trajectory", exist_ok=True)
os.makedirs("Users/<username>/results/Communications", exist_ok=True)
os.makedirs("Users/<username>/results/Proportions", exist_ok=True)

import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc

def split_umap(adata, split_by, basis='X_umap', color_by=None, ncol=2, nrow=None, save_path=None, dpi=300, **kwargs):
    categories = adata.obs[split_by].cat.categories
    if nrow is None:
        nrow = int(np.ceil(len(categories) / ncol))

    fig, axs = plt.subplots(nrow, ncol, figsize=(5*ncol, 4*nrow))
    axs = axs.flatten()
    for i, cat in enumerate(categories):
        ax = axs[i]
        sc.pl.embedding(
            adata,
            basis=basis,
            color=color_by if color_by else split_by,
            groups=[cat],
            ax=ax,
            show=False,
            title=cat,
            **kwargs
        )

    for j in range(i+1, len(axs)):
        fig.delaxes(axs[j])

    plt.tight_layout()
    if save_path:
        fig.savefig(save_path, dpi=dpi, bbox_inches='tight')
        print(f"Saved figure to {save_path}")

    plt.show()


from adjustText import adjust_text
import matplotlib.pyplot as plt

def gen_mpl_labels(
    adata,
    groupby,
    basis="X_umap",           
    exclude=(),
    ax=None,
    adjust_kwargs=None,
    text_kwargs=None,
    color_by_group=False,
    show_labels=True,
):
    if adjust_kwargs is None:
        adjust_kwargs = {"text_from_points": False}
    if text_kwargs is None:
        text_kwargs = {}

    if basis not in adata.obsm_keys():
        raise ValueError(f"{basis} non trovato in adata.obsm!")

    coords = adata.obsm[basis]

    medians = {}
    for g in adata.obs[groupby].cat.categories:
        if g in exclude:
            continue
        mask = adata.obs[groupby] == g
        if mask.sum() == 0:
            continue
        group_coords = coords[mask.values]
        medians[g] = np.median(group_coords, axis=0)

    text_colors = {g: "black" for g in adata.obs[groupby].cat.categories}
    if color_by_group and f"{groupby}_colors" in adata.uns:
        for i, g in enumerate(adata.obs[groupby].cat.categories):
            if g in exclude:
                continue
            text_colors[g] = adata.uns[f"{groupby}_colors"][i]

    texts = []
    if show_labels:
        for g, (x, y) in medians.items():
            txt = (ax if ax is not None else plt).text(
                x, y, g, color=text_colors[g], **text_kwargs
            )
            texts.append(txt)

        adjust_text(texts, **adjust_kwargs)

    return texts


def process(adata, resolution=0.4):    
    if adata.X.toarray().max() > 100:
        adata.X = adata.layers["raw_counts"].copy()
        sc.pp.normalize_total(adata, target_sum=10**4) 
    
        sc.pp.log1p(adata)
    else:
        print("Data are already log trasformed")
    sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
    if adata.raw is None:
        print("log1p data before filtering have been saved in .raw.X")
        adata.raw = adata
    adata.layers["log1p"] = adata.X.copy()
   
    adata = adata[:, adata.var['highly_variable']]
    sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])
    sc.pp.scale(adata, max_value=10)
    sc.tl.pca(adata, svd_solver='arpack', random_state=1)
    sc.pp.neighbors(adata, random_state=1)
    sc.tl.leiden(adata, resolution=resolution, random_state=1, key_added= f"leiden_{resolution}")
    sc.tl.umap(adata, random_state=1)
    
    return adata


def do_harmony(adata_to_harmonize, resolution=0.4, batch_column='orig.ident'):
    adata = adata_to_harmonize.copy()
    adata.obs['batch'] = adata.obs[batch_column].astype(str)
    adata.uns['orig_umap'] = adata.uns['umap']
    adata.obsm['orig_X_umap'] = adata.obsm['X_umap']
    ho = hm.run_harmony(
        adata.obsm['X_pca'],     
        adata.obs,               
        'batch', random_state=42  
    )
    adata.obsm[f'X_harmony_{batch_column}'] = ho.Z_corr.T

    sc.pp.neighbors(adata, random_state=42, use_rep=f'X_harmony_{batch_column}', key_added=f'harmony_nn_{batch_column}')
    sc.tl.leiden(adata, random_state=42, resolution=resolution, neighbors_key=f'harmony_nn_{batch_column}', key_added=f"harmony_leiden_{batch_column}_{resolution}")
    sc.tl.umap(adata, random_state=42, neighbors_key=f'harmony_nn_{batch_column}')

    adata.uns[f'harmony_umap_{batch_column}'] = adata.uns['umap']
    adata.obsm[f'X_harmony_umap_{batch_column}'] = adata.obsm['X_umap']

    adata.uns['umap'] = adata.uns['orig_umap']
    adata.obsm['X_umap'] = adata.obsm['orig_X_umap']
    del adata.uns['orig_umap']
    del adata.obsm['orig_X_umap']

    return adata

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


## Set Environment 

Change the directory you are working on into the directory containing the data

In [ ]:
print("directory now: ", os.getcwd())
os.chdir("Users/<username>/data/")
print("directory now: ", os.getcwd())
print("directory files: ")
for file in os.listdir():
    print(f"\n{file}")
palette = sns.color_palette("husl", 36)
res = 1

## Process and perform integration

In [ ]:
os.chdir("Users/<username>/data/")
if "AdataNorm" not in os.listdir():
        adata =sc.read_h5ad(
        "AdataFiltered")
        adata.layers["raw_counts"]=adata.X.copy()
        adata_sel=process(adata, resolution= 1)
        
        if not adata_sel.obs_names.is_unique:
            new_names = [
            f"{barcode}_{patient}"
            for barcode, patient in zip(adata_sel.obs_names, adata_sel.obs["orig.ident"])
            ]
            adata_sel.obs_names = new_names
        import openpyxl
        file_path2 = "ptsinfo.xlsx"
        df_clinici2 = pd.read_excel(file_path2)

        df_clinici2 = df_clinici2[df_clinici2['orig.ident'].notna()]
        df_clinici2 = df_clinici2.replace(r'^\s*$', np.nan, regex=True)
        df_clinici2 = df_clinici2.dropna(axis=0, how='all')
        df_clinici2 = df_clinici2.dropna(thresh=round(len(df_clinici2) * 0.7), axis=1)
        cols = ['orig.ident'] + [c for c in df_clinici2.columns if c != 'orig.ident']
        df_clinici2 = df_clinici2[cols]
        df_clinici2 = df_clinici2.astype('category')
        df_clinici2 = df_clinici2.applymap(lambda x: x.strip() if isinstance(x, str) else x)
        df_clinici2.columns = df_clinici2.columns.str.strip()
        df_clinici2 = df_clinici2[df_clinici2["orig.ident"].isin(adata_sel.obs["orig.ident"].value_counts().keys())]
        df_clinici2 = df_clinici2.set_index("orig.ident")
        adata_sel.obs["Condition"] = adata_sel.obs["orig.ident"].map(df_clinici2["Condition"])
        adata_sel = do_harmony(adata_sel,resolution=1, batch_column='orig.ident')
        adata_sel = do_harmony(adata_sel,resolution=1, batch_column='Condition_site_T0')
        adata_sel.write_h5ad(
                "AdataNorm",
                compression=hdf5plugin.FILTERS["zstd"])
else:
    adata =sc.read_h5ad(
        "AdataFiltered")
    adata_sel=sc.read_h5ad(
        "AdataNorm")

# Automated Annotation

Create the custom model to perform automated annotation with celltypist

Here we:
1. Create a copy of the processed Anndata without the highly variable genes mask in order to be compatible with celltypist
2. Make the predictions on the New Anndata and transfer them into the previous one
3. Overwrite the data as "AdataNorm" to save computational time in the future

In [38]:
adata_sel

AnnData object with n_obs × n_vars = 106458 × 3195
    obs: 'orig.ident', 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_Erythroid', 'log1p_total_counts_Erythroid', 'pct_counts_Erythroid', 'outlier', 'mt_outlier', 'erythroid_outlier', 'n_genes_by_counts_outlier', 'scDblFinder_score', 'scDblFinder_class', 'leiden_1', 'Biopsy_site_T0', 'batch', 'harmony_leiden_orig.ident_1', 'harmony_leiden_Biopsy_site_T0_1', 'MBC_Atlas_pred', 'MBC_Atlas_score', 'Immhigh_pred', 'Immhigh_score', 'ImmLow_pred', 'ImmLow_score'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'harmony_leiden_Biopsy_site_T0_1', 'harmony_leiden_orig.ident_1', 'harmony_nn_Biopsy_site_T0', 'harmony_nn_orig.ident', 'harmony_umap_Biopsy_site_T0', 'harmony_umap_orig.id

In [ ]:
if "MBC_Atlas_pred" not in adata_sel.obs.columns:
    adata_sel =do_harmony(adata_sel, resolution=res, batch_column='orig.ident') #Condition_site is not good enough
    adata_sel.uns['umap'] = adata_sel.uns['harmony_umap_orig.ident'].copy()
    adata_sel.uns['neighbors'] = adata_sel.uns['harmony_nn_orig.ident'].copy()
    adata_sel.obsp['distances'] = adata_sel.obsp[ 'harmony_nn_orig.ident_distances'].copy()
    adata_sel.obsp['connectivities'] = adata_sel.obsp['harmony_nn_orig.ident_connectivities'].copy()
    adata_sel.obsm['X_umap'] = adata_sel.obsm['X_harmony_umap_orig.ident'].copy()
    adata_sel.obsm['X_pca'] = adata_sel.obsm['X_harmony_orig.ident'].copy()
    adata_sel.obs["leiden_1"] = adata_sel.obs["harmony_leiden_orig.ident_1"].copy()
    
    adata_complete=anndata.AnnData(X=adata_sel.raw.X
                          ,obs=adata_sel.obs
                          ,var=adata_sel.raw.var
                          ,uns=adata_sel.uns
                          ,obsm=adata_sel.obsm
                          ,obsp=adata_sel.obsp)
    adata_complete.X = adata_complete.X.toarray()
    predictions = celltypist.annotate(adata_complete, model = 'Immune_All_High.pkl', majority_voting = True,  mode = 'prob match', p_thres = 0.6)
    adata_pred = predictions.to_adata()
    adata_sel.obs["Immhigh_pred"] = adata_pred.obs.loc[
        adata_sel.obs.index, "majority_voting"
    ]
    adata_sel.obs["Immhigh_score"] = adata_pred.obs.loc[
        adata_sel.obs.index, "conf_score"
    ]   
    del adata_pred

    predictions = celltypist.annotate(adata_complete, model = 'Immune_All_Low.pkl', majority_voting = True,  mode = 'prob match', p_thres = 0.6)
    adata_pred = predictions.to_adata()
    adata_sel.obs["ImmLow_pred"] = adata_pred.obs.loc[
        adata_sel.obs.index, "majority_voting"
    ]
    adata_sel.obs["ImmLow_score"] = adata_pred.obs.loc[
        adata_sel.obs.index, "conf_score"
    ]
    del adata_pred
    del adata_complete
    #convert categories as strings
    for col in adata_sel.obs.select_dtypes(['category']).columns:
        adata_sel.obs[col] = adata_sel.obs[col].astype(str)
    adata_sel.write_h5ad(
            "AdataNorm",
            compression=hdf5plugin.FILTERS["zstd"])


In [ ]:
adata_sel.obs["median_score"] = adata_sel.obs[["Immhigh_score","ImmLow_score"]].median(axis=1)
adata_sel.obs["mean_score"] = adata_sel.obs[["Immhigh_score","ImmLow_score"]].mean(axis=1)

print( "Min mean score:" , round(adata_sel.obs["mean_score"].min(),3))
print( "Max mean score:" , round(adata_sel.obs["mean_score"].max(),3))

Plot the scores and the predictions

In [ ]:
sc.pl.umap(adata_sel, color=["Immhigh_score","ImmLow_score", "mean_score"])


In [ ]:
sc.pl.umap(adata_sel, color=['n_genes_by_counts'
                             , 'log1p_n_genes_by_counts'
                             , 'log1p_total_counts'
                             , 'pct_counts_mt'
                             , 'pct_counts_ribo'
                             , 'pct_counts_Erythroid'
                             ]
                             , ncols= 1)

# Commonly used Markers

Visualize commonly known Markers

In [ ]:


res_dir = "Users/<username>/results/"
fig = sc.pl.embedding(
    adata_sel,
    basis='X_umap',
    color=[
            'EPCAM','KRT5','KRT7','KRT8','KRT14','KRT18','KRT16','KRT17','CCN1', 'DST','TRPS1', 'CCSER1','ESR1','ERBB2', 'MKI67',
            'PECAM1', 'PROX1', 'CD34',
            'GNLY', 'NKG7', 'CD247',
            'CD3D', 'CD3E',
            'MS4A1',
            'JCHAIN','CLEC4C','LILRA4',
            'CD68',
            'KIT',
            'PDGFRA','PDGFRB', 
            'COL1A1', 
            'COL3A1', 
            'DCN',
            'LUM',
            'ACTA2','RGS5',
            'ALB', 'APOB','FABP1', 'AFP'],
    ncols=3,
    vmin=0,
    vmax="p99",         
    sort_order=False,   
    cmap="viridis",
    return_fig=True,
    wspace=0.5
)


fig.savefig(res_dir+'Markers_Plots/PreFiltering/KnownMarkersRAW.pdf',bbox_inches='tight')



# Manual Curation

In [46]:
res = 2
sc.tl.leiden(adata_sel, resolution=res, random_state=1, key_added= f"leiden_{res}")

## Plot clusters

In [ ]:
import matplotlib.patheffects as pe
plt.rcParams.update({'font.size': 10})
with plt.rc_context({"figure.figsize": (12, 8), "figure.dpi": 150, "figure.frameon": False}):
    fig = sc.pl.embedding(
    adata_sel,
    basis= "X_umap",
    color=f"leiden_{res}",
    show=False,
    legend_loc='right margin',
    frameon=False,
    size=5,
    add_outline=True,
    return_fig=True
    )
    ax = fig.axes[0]
    gen_mpl_labels(
    adata_sel,
    groupby=f"leiden_{res}",
    basis= "X_umap",
    ax=ax,
    text_kwargs=dict(fontsize=8, path_effects=[pe.withStroke(linewidth=4, foreground="black")]),
    adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
    color_by_group=True
    )
    gen_mpl_labels(
    adata_sel,
    groupby=f"leiden_{res}",
    basis= "X_umap",
    ax=ax,
    text_kwargs=dict(fontsize=8, path_effects=[pe.withStroke(linewidth=3, foreground="white")]),
    adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
    color_by_group=True
    )

    fig.tight_layout()
    fig.savefig("Users/<username>/results/Markers_Plots/PreFiltering/ClustersRAW.png", dpi=300, bbox_inches="tight")
    plt.show()
    plt.close(fig)

## Plot predictions and Biological conditions

In [ ]:
#predictions and/or biological conditions can hel with the classifications
sc.pl.umap(adata_sel, color=["Condition"
                             , "Immhigh_pred"
                             , "ImmLow_pred"]
                             , ncols= 1)

## Perform Differential gene expression analysis

In [ ]:
adata = adata_sel.copy()
sc.tl.rank_genes_groups(
    adata, groupby=f"leiden_{res}"
    , method="wilcoxon"
    , key_added="deg_clusters"
)


sc.tl.filter_rank_genes_groups(
    adata,
    min_fold_change=1,
    min_in_group_fraction=0.6,
    max_out_group_fraction=0.4,
    key="deg_clusters",
    key_added="deg_clusters_filtered",
)

out = "Users/<username>/results/Markers_Plots/cells_DGE.pdf"
os.makedirs(os.path.dirname(out), exist_ok=True)

with plt.rc_context({'font.size': 18}):
    dp = sc.pl.rank_genes_groups_dotplot(
    adata,
    groupby=f"leiden_{res}",
    # standard_scale="var",
    vmin=0,
    vmax=4,
    n_genes=3,
    key="deg_clusters_filtered",
    cmap='viridis',#'RdBu_r',  # "viridis",
    #figsize=(60, 8),
    min_logfoldchange=1,
    #values_to_plot="logfoldchanges",
    #colorbar_title='log fold change',
    colorbar_title='Mean expression',
    show=True,  # Importante per ulteriori modifiche
    return_fig=True,
    swap_axes=True,
    save = f"_leiden{res}_DGE.pdf")
    dp.legends_width = 3        # <— allarga lo spazio delle legende
    dp.savefig(out, bbox_inches="tight")

## Rename clusters accordingly

In [ ]:
# Rename every cluster according to Automated annotation, common markers and DGE
# in this example we use 20 clusters but the number of clusters is experiment dependant
plt.rcParams.update({'font.size': 6})
adata_sel.obs["cell_type"] = adata_sel.obs[f"leiden_{res}"].map(
    {
        "0": "<insert cell type here>",
        "1": "<insert cell type here>",
        "2": "<insert cell type here>",
        "3": "<insert cell type here>",
        "4": "<insert cell type here>",
        "5": "<insert cell type here>",
        "6": "<insert cell type here>",
        "7": "<insert cell type here>",
        "8": "<insert cell type here>",
        "9": "<insert cell type here>",
        "10": "<insert cell type here>",
        "11": "<insert cell type here>",
        "12": "<insert cell type here>",
        "13": "<insert cell type here>",
        "14": "<insert cell type here>",
        "15": "<insert cell type here>",
        "16": "<insert cell type here>",
        "17": "<insert cell type here>",
        "18": "<insert cell type here>",
        "19": "<insert cell type here>",
        "20": "To delete" # mark unknown / unwanted clusters with the To delete notation 
    }
)



adata_sel.obs["Condition"] = (
    adata_sel.obs["Condition"]
    .astype(str)   
    .str.strip()   
    .astype("category")  
)
adata_sel.obs["cell_type"] = adata_sel.obs["cell_type"].astype(str)
adata_sel.obs["Condition"] = adata_sel.obs["Condition"].astype(str)
print( pd.value_counts(adata_sel.obs["cell_type"]), adata_sel.n_obs )

ax = sc.pl.embedding(adata_sel, color="cell_type", legend_loc='right margin', frameon=False
                    , size = 1, add_outline= True,title=" ", basis="X_umap")
adata_sel = adata_sel[adata_sel.obs["cell_type"] != "To delete"].copy()

## Plot final annotation

In [ ]:
import matplotlib.patheffects as pe
plt.rcParams.update({'font.size': 10})
with plt.rc_context({"figure.figsize": (12, 8), "figure.dpi": 150, "figure.frameon": False}):
    fig = sc.pl.embedding(
    adata_sel,
    basis= "X_umap",
    color="cell_type",
    show=False,
    legend_loc='right margin',
    frameon=False,
    size=5,
    add_outline=True,
    return_fig=True
    )
    ax = fig.axes[0]
    gen_mpl_labels(
    adata_sel,
    groupby="cell_type",
    basis= "X_umap",
    ax=ax,
    text_kwargs=dict(fontsize=8, path_effects=[pe.withStroke(linewidth=4, foreground="black")]),
    adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
    color_by_group=True
    )
    gen_mpl_labels(
    adata_sel,
    groupby="cell_type",
    basis= "X_umap",
    ax=ax,
    text_kwargs=dict(fontsize=8, path_effects=[pe.withStroke(linewidth=3, foreground="white")]),
    adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
    color_by_group=True
    )

    fig.tight_layout()
    fig.savefig("Users/<username>/results/Markers_Plots/PreFiltering/CellsRAW.png", dpi=300, bbox_inches="tight")
    plt.show()
    plt.close(fig)

## Redo processing but only on known cells

In the annotation process one can filter out multiple cells. The intrinsic variation of the dataset can be influenced by that.

 For this reason we have to redo the normalization process and the UMAP construction

In [54]:
#reload the AdataFiltered, subset it with adata_sel, use process functiomn
adata =sc.read_h5ad(
        "AdataFiltered")
adata.layers["raw_counts"]=adata.X.copy()

if not adata.obs_names.is_unique:
    adata.obs["orig.ident"] = adata.obs["orig.ident"].str.replace('_SF', '', regex=False)
    # 1) Crea una nuova lista di nomi basata sull'indice e su "orig.ident"
    new_names = [
     f"{barcode}_{patient}"
     for barcode, patient in zip(adata.obs_names, adata.obs["orig.ident"])
    ]

# 2) Assegna la nuova lista di nomi come indice
adata.obs_names = new_names
adata = adata[adata_sel.obs_names].copy()
adata.obs = adata_sel.obs
adata_selOLD = adata_sel.copy()
del adata_sel
adata_sel =process(adata, resolution= 0.7)
adata_sel = do_harmony(adata_sel,resolution=1, batch_column='orig.ident')


log1p data before filtering have been saved in .raw.X


2025-11-03 14:29:14,540 - harmonypy - INFO - Computing initial centroids with sklearn.KMeans...
2025-11-03 14:29:14 | [INFO] Computing initial centroids with sklearn.KMeans...
2025-11-03 14:29:21,391 - harmonypy - INFO - sklearn.KMeans initialization complete.
2025-11-03 14:29:21 | [INFO] sklearn.KMeans initialization complete.
2025-11-03 14:29:21,567 - harmonypy - INFO - Iteration 1 of 10
2025-11-03 14:29:21 | [INFO] Iteration 1 of 10
2025-11-03 14:29:37,350 - harmonypy - INFO - Iteration 2 of 10
2025-11-03 14:29:37 | [INFO] Iteration 2 of 10
2025-11-03 14:29:53,052 - harmonypy - INFO - Iteration 3 of 10
2025-11-03 14:29:53 | [INFO] Iteration 3 of 10
2025-11-03 14:30:08,494 - harmonypy - INFO - Iteration 4 of 10
2025-11-03 14:30:08 | [INFO] Iteration 4 of 10
2025-11-03 14:30:23,632 - harmonypy - INFO - Iteration 5 of 10
2025-11-03 14:30:23 | [INFO] Iteration 5 of 10
2025-11-03 14:30:38,900 - harmonypy - INFO - Converged after 5 iterations
2025-11-03 14:30:38 | [INFO] Converged after 5

## Plot the new UMAP

Here we just plot the new obtained UMAP with the cell clusters, annotations, adn canonical markers.


In [ ]:
import matplotlib.patheffects as pe
plt.rcParams.update({'font.size': 10})
with plt.rc_context({"figure.figsize": (12, 8), "figure.dpi": 150, "figure.frameon": False}):
    fig = sc.pl.embedding(
    adata_sel,
    basis= 'X_harmony_umap_orig.ident',
    color="harmony_leiden_orig.ident_0.7",
    show=False,
    legend_loc='right margin',
    frameon=False,
    size=5,
    add_outline=True,
    return_fig=True
    )
    ax = fig.axes[0]
    gen_mpl_labels(
    adata_sel,
    groupby="harmony_leiden_orig.ident_0.7",
    basis= 'X_harmony_umap_orig.ident',
    ax=ax,
    text_kwargs=dict(fontsize=8, path_effects=[pe.withStroke(linewidth=4, foreground="black")]),
    adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
    color_by_group=True
    )
    gen_mpl_labels(
    adata_sel,
    groupby="harmony_leiden_orig.ident_0.7",
    basis= 'X_harmony_umap_orig.ident',
    ax=ax,
    text_kwargs=dict(fontsize=8, path_effects=[pe.withStroke(linewidth=3, foreground="white")]),
    adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
    color_by_group=True
    )

    fig.tight_layout()
    fig.savefig("Users/<username>/results/Markers_Plots/Umap/Clusters.png", dpi=300, bbox_inches="tight")
    plt.show()
    plt.close(fig)

In [ ]:
res_dir = "Users/<username>/results/"
fig = sc.pl.embedding(adata_sel, basis='X_harmony_umap_orig.ident',
        color= ['EPCAM','KRT5','KRT7','KRT8','KRT14','KRT18','KRT16','KRT17','CCN1', 'DST','TRPS1', 'CCSER1','ESR1','ERBB2', 'MKI67',
        'PECAM1', 'PROX1', 'CD34',
        'GNLY', 'NKG7', 'CD247',
        'CD3D', 'CD3E',
        'MS4A1',
        'JCHAIN','CLEC4C','LILRA4',
        'CD68',
        'KIT',
        'PDGFRA','PDGFRB', 
        'COL1A1', 
        'COL3A1', 
        'DCN',
        'LUM',
        'ACTA2','RGS5',
        'ALB', 'APOB','FABP1', 'AFP'],
        ncols = 3,
        vmin=0,
        vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
        sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
        frameon=False,
        cmap="viridis", return_fig=True,wspace=0.5)  # or choose another color map e.g. from here: https://matplotlib.org/stable/tutorials/colors/colormaps.html )

fig.savefig(res_dir+'Markers_Plots/Umap/KnownMarkers.png',bbox_inches='tight')

In [ ]:
import matplotlib.patheffects as pe
plt.rcParams.update({'font.size': 10})
with plt.rc_context({"figure.figsize": (12, 8), "figure.dpi": 150, "figure.frameon": False}):
    fig = sc.pl.embedding(
    adata_sel,
    basis= 'X_harmony_umap_orig.ident',
    color="cell_type",
    show=False,
    legend_loc='right margin',
    frameon=False,
    size=5,
    add_outline=True,
    return_fig=True
    )
    ax = fig.axes[0]
    gen_mpl_labels(
    adata_sel,
    groupby="cell_type",
    basis= 'X_harmony_umap_orig.ident',
    ax=ax,
    text_kwargs=dict(fontsize=8, path_effects=[pe.withStroke(linewidth=4, foreground="black")]),
    adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
    color_by_group=True
    )
    gen_mpl_labels(
    adata_sel,
    groupby="cell_type",
    basis= 'X_harmony_umap_orig.ident',
    ax=ax,
    text_kwargs=dict(fontsize=8, path_effects=[pe.withStroke(linewidth=3, foreground="white")]),
    adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
    color_by_group=True
    )

    fig.tight_layout()
    fig.savefig("Users/<username>/results/info_Plots/Harmony/CellsHRM.png", dpi=300, bbox_inches="tight")
    plt.show()
    plt.close(fig)

In [ ]:

import matplotlib.patheffects as pe
plt.rcParams.update({'font.size': 10})
with plt.rc_context({"figure.figsize": (12, 8), "figure.dpi": 150, "figure.frameon": False}):
    fig = sc.pl.embedding(
    adata_sel,
    basis= 'X_harmony_umap_orig.ident',
    color="Condition",
    show=False,
    legend_loc='right margin',
    frameon=False,
    size=5,
    add_outline=True,
    return_fig=True
    )

    fig.tight_layout()
    fig.savefig("Users/<username>/results/info_Plots/Harmony/Condition_HRM.pdf", dpi=300, bbox_inches="tight")
    plt.show()
    plt.close(fig)

In [ ]:
split_umap(adata_sel,  basis='X_harmony_umap_orig.ident', split_by="Condition", ncol= 4, na_in_legend = False, save_path="Users/<username>/results/info_Plots/Harmony/splitCondition_HRM.pdf")


In [ ]:
os.chdir("Users/<username>/data/")
adata_sel.obs["Cells"] = adata_sel.obs["cell_type"]
adata_sel.write_h5ad(
            "AdataFinal",
            compression=hdf5plugin.FILTERS["zstd"])

In [ ]:
os.chdir("Users/<username>/data/")
adata_sel = sc.read_h5ad( "AdataFinal")

In [81]:
adata_sel.obs.Cells.value_counts()

Cells
MBC              50820
Fibroblasts      17942
VECs              6707
T cells CD4+      6147
Pericytes         5830
T cells CD8+      4462
Macrophages       3882
Keratinocytes     2798
Mast cells        1110
B cells           1048
Monocytes         1031
NK cells           772
LECs               573
Hepatocytes        383
Name: count, dtype: int64

## Final DE analysis to double check Cells identity

In [ ]:
adata = adata_sel.copy()
sc.tl.rank_genes_groups(
    adata, groupby=f"Cells"
    , method="wilcoxon"
    , key_added="deg_clusters"
)


sc.tl.filter_rank_genes_groups(
    adata,
    min_fold_change=1,
    min_in_group_fraction=0.6,
    max_out_group_fraction=0.4,
    key="deg_clusters",
    key_added="deg_clusters_filtered",
)

out = "Users/<username>/results/Markers_Plots/cells_DGE.pdf"
os.makedirs(os.path.dirname(out), exist_ok=True)

with plt.rc_context({'font.size': 18}):
    dp = sc.pl.rank_genes_groups_dotplot(
    adata,
    groupby=f"Cells",
    # standard_scale="var",
    vmin=0,
    vmax=4,
    n_genes=3,
    key="deg_clusters_filtered",
    cmap='viridis',#'RdBu_r',  # "viridis",
    #figsize=(60, 8),
    min_logfoldchange=1,
    #values_to_plot="logfoldchanges",
    #colorbar_title='log fold change',
    colorbar_title='Mean expression',
    show=True,  
    return_fig=True,
    swap_axes=True,
    save = "_Cells_DGE.pdf")
    dp.legends_width = 3        
    dp.savefig(out, bbox_inches="tight")